In [1]:
import keras
import numpy as np 
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten,LeakyReLU

Using TensorFlow backend.


In [2]:
NUM_CLASSES = 7
CLASSES = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [3]:
model = Sequential()
model.add(Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(64,64,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7,activation='sigmoid'))

In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
training_set = train_datagen.flow_from_directory('emotion/train',target_size = (64, 64),batch_size = 32)

test_datagen = ImageDataGenerator()
test_set = test_datagen.flow_from_directory(directory='emotion/test',target_size = (64, 64), batch_size = 32)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
model.fit(training_set,validation_data=test_set,epochs=25)

Epoch 1/25
898/898 [==============================] - 257s 286ms/step - loss: 0.6123 - accuracy: 0.8527 - val_loss: 0.3327 - val_accuracy: 0.8606
Epoch 2/25
898/898 [==============================] - 273s 304ms/step - loss: 0.3741 - accuracy: 0.8599 - val_loss: 0.3868 - val_accuracy: 0.8650
Epoch 3/25
898/898 [==============================] - 328s 365ms/step - loss: 0.3628 - accuracy: 0.8624 - val_loss: 0.3880 - val_accuracy: 0.8716
Epoch 4/25
898/898 [==============================] - 338s 377ms/step - loss: 0.3532 - accuracy: 0.8649 - val_loss: 0.3350 - val_accuracy: 0.8728
Epoch 5/25
898/898 [==============================] - 299s 333ms/step - loss: 0.3421 - accuracy: 0.8682 - val_loss: 0.3373 - val_accuracy: 0.8749
Epoch 6/25
898/898 [==============================] - 311s 346ms/step - loss: 0.3346 - accuracy: 0.8705 - val_loss: 0.2318 - val_accuracy: 0.8774
Epoch 7/25
898/898 [==============================] - 295s 329ms/step - loss: 0.3277 - accuracy: 0.8724 - val_loss: 0.2993 -

In [16]:
model.save('model.h5')

In [5]:
recogniser = keras.models.load_model('model.h5')

In [6]:
recogniser.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 62, 62, 16)        448       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 60, 60, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 26, 26, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 64)       

In [8]:
import cv2
def face(path):
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    img = cv2.imread(path)
    img = cv2.resize(img,(512,512))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray,1.1,4)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)

    img= gray[y-32:y+h+32,x-32:x+w+32]
    cv2.imwrite('image.png',img)

In [9]:
# for recognising people in front of camera
cap = cv2.VideoCapture(0)
_ , frame = cap.read()
cv2.imwrite('image.png',frame)
cap.release()
cv2.destroyAllWindows()

In [10]:
face('image.png')
# cv2.imshow('image',img)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [11]:
image = keras.preprocessing.image.load_img('image.png',target_size=[64,64,3])
image=keras.preprocessing.image.img_to_array(image)
image=np.expand_dims(image,axis=0)

In [12]:
result = recogniser.predict(image)
result

array([[0.20050837, 0.0028386 , 0.14539558, 0.00728727, 0.34907788,
        0.29915527, 0.02225633]], dtype=float32)

In [13]:
max_prob_index = np.argmax(result)
CLASSES[max_prob_index]


'neutral'